In [9]:
import pandas as pd
from sklearn.model_selection import KFold,train_test_split
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.ensemble import RandomForestRegressor
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV


In [10]:
sheet_name='Sheet3'
data= pd.read_excel("F:/Thesis/Nazmul sir/DATA/bangladesh-inflation-rate-cpi.xlsx", sheet_name=sheet_name)
print(data.head())
""" plt.plot(data['Year'],data['Exchange rate'],'o-')
plt.show() """

        Date  Exchange rate
0 1996-10-09          42.45
1 1996-10-10          42.45
2 1996-10-11          42.45
3 1996-10-14          42.45
4 1996-10-15          42.45


" plt.plot(data['Year'],data['Exchange rate'],'o-')\nplt.show() "

In [11]:
for i in range(1,8):
    data[f'Lag{i}']=data['Exchange rate'].shift(i)
data.dropna(inplace=True)
data['Day_of_Week'] = data['Date'].dt.day
data['Month'] = data['Date'].dt.month
data['Year'] = data['Date'].dt.year
print(data.head())

         Date  Exchange rate   Lag1   Lag2   Lag3   Lag4   Lag5   Lag6   Lag7  \
7  1996-10-18          42.45  42.45  42.45  42.45  42.45  42.45  42.45  42.45   
8  1996-10-21          42.45  42.45  42.45  42.45  42.45  42.45  42.45  42.45   
9  1996-10-22          42.45  42.45  42.45  42.45  42.45  42.45  42.45  42.45   
10 1996-10-23          42.45  42.45  42.45  42.45  42.45  42.45  42.45  42.45   
11 1996-10-24          42.45  42.45  42.45  42.45  42.45  42.45  42.45  42.45   

    Day_of_Week  Month  Year  
7            18     10  1996  
8            21     10  1996  
9            22     10  1996  
10           23     10  1996  
11           24     10  1996  


In [12]:
x=data.drop(columns=['Date','Exchange rate','Year'])
y=data['Exchange rate']
print(x.head())
print(y.head())
print(y.dtype)



     Lag1   Lag2   Lag3   Lag4   Lag5   Lag6   Lag7  Day_of_Week  Month
7   42.45  42.45  42.45  42.45  42.45  42.45  42.45           18     10
8   42.45  42.45  42.45  42.45  42.45  42.45  42.45           21     10
9   42.45  42.45  42.45  42.45  42.45  42.45  42.45           22     10
10  42.45  42.45  42.45  42.45  42.45  42.45  42.45           23     10
11  42.45  42.45  42.45  42.45  42.45  42.45  42.45           24     10
7     42.45
8     42.45
9     42.45
10    42.45
11    42.45
Name: Exchange rate, dtype: float64
float64


In [13]:
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

# Define the parameter grid for GridSearchCV
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Perform GridSearchCV
grid_search = GridSearchCV(estimator=RandomForestRegressor(random_state=42),
                           param_grid=param_grid,
                           cv=5,
                           scoring='neg_mean_squared_error',
                           n_jobs=-1)
grid_search.fit(x, y)

# Get the best hyperparameters
best_params = grid_search.best_params_

# Initialize list to store validation RMSE
val_rmse_list = []
val_r2=[]

# Initialize KFold
kf = KFold(n_splits=100, shuffle=True, random_state=42)

x_train,x_test,y_train,y_test= train_test_split(x,y,train_size=.8,random_state=42)

# Perform k-fold cross-validation
for train_index, val_index in kf.split(x_train):
    x_train_fold, x_val_fold = x_train.iloc[train_index], x_train.iloc[val_index]
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

    # Create and train RandomForestRegressor with best hyperparameters
    rf_model = RandomForestRegressor(**best_params, random_state=42)
    rf_model.fit(x_train_fold, y_train_fold)
    
    # Make predictions on validation set
    y_pred_val = rf_model.predict(x_val_fold)

    # Calculate validation RMSE
    val_rmse = mean_squared_error(y_val_fold, y_pred_val, squared=False)
    val_rmse_list.append(val_rmse)
    r2=r2_score(y_val_fold,y_pred_val)
    val_r2.append(r2)

# Calculate average model RMSE
mean_rmse = np.mean(val_rmse_list)

# Print results
print("Average model RMSE:", mean_rmse)
print("Validation RMSE for each fold:", val_rmse_list)
print (best_params)
print(f"Average r2 score is {np.mean(val_r2)}")


Average model RMSE: 0.2834155874746977
Validation RMSE for each fold: [0.4995621569622701, 0.3354811494424842, 0.25756074105830984, 0.17778369267698851, 0.16729028041925686, 0.16445653182135242, 0.1432811121478327, 0.28987996553662143, 0.20409769394013022, 0.9601229037356692, 0.3646511660870282, 0.29228231329962046, 0.23925204638386566, 0.2768551840722961, 0.4266077019254967, 0.3015431576638636, 0.34286167358448505, 0.19231506117653413, 0.2608171925975863, 0.22322867002096497, 0.1976682405106213, 0.1736532404859691, 0.25337670579292293, 0.22633250235660582, 0.1309535260406707, 0.18887909228003194, 0.3963014845205024, 0.24915505162914087, 0.1697808144818908, 0.36694854744791183, 0.3733216867010993, 0.17894097687853924, 0.17179805406061957, 0.35194824594515667, 0.3286367804883639, 0.4366710351359646, 0.28210456160808595, 0.11007709430412506, 0.25251212910674503, 0.13907446087549843, 0.3328036575803073, 0.5824722669817255, 0.2562501565011648, 0.19437328375487636, 0.2256965852425174, 0.107

In [14]:
Final_model=RandomForestRegressor(max_depth=10,min_samples_leaf=4,min_samples_split=2,n_estimators=50,random_state=42)
Final_model.fit(x_train,y_train)
Prediction=Final_model.predict(x_test)
RMSE=mean_squared_error(Prediction,y_test,squared=False)
print(f'Model RMSE value is: {RMSE}')

Model RMSE value is: 0.2992510825344234


In [15]:
""" y_numeric = pd.to_numeric(y, errors='coerce')

import matplotlib.pyplot as plt

# Make predictions on the entire dataset
y_pred = rf_model.predict(x)

# Plot predicted vs. actual values using y_numeric
plt.figure(figsize=(8, 6))
plt.scatter(y_numeric, y_pred, color='blue', alpha=0.5)
plt.plot([y_numeric.min(), y_numeric.max()], [y_numeric.min(), y_numeric.max()], color='red', linestyle='--', lw=2)
plt.title('Predicted vs. Actual Values')
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.grid(True)
plt.show()
 """

" y_numeric = pd.to_numeric(y, errors='coerce')\n\nimport matplotlib.pyplot as plt\n\n# Make predictions on the entire dataset\ny_pred = rf_model.predict(x)\n\n# Plot predicted vs. actual values using y_numeric\nplt.figure(figsize=(8, 6))\nplt.scatter(y_numeric, y_pred, color='blue', alpha=0.5)\nplt.plot([y_numeric.min(), y_numeric.max()], [y_numeric.min(), y_numeric.max()], color='red', linestyle='--', lw=2)\nplt.title('Predicted vs. Actual Values')\nplt.xlabel('Actual Values')\nplt.ylabel('Predicted Values')\nplt.grid(True)\nplt.show()\n "